<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/%E4%BD%9C%E6%A5%AD1_0918.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日常支出速算與分攤（作業一）
*   目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
*   AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
*   Sheet 欄位：date,time,item,money,category,pay_way
*   googlesheet:https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing
*   Gradio：https://7862-m-s-1p1etdre2pkv5-b.us-east1-1.prod.colab.dev/






In [ ]:
!pip install gspread oauth2client

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import pandas as pd
import datetime
#from types import new_class
import gradio as gr

creds,_ = default()
gc = gspread.authorize(creds)
gsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing')
worksheet = gsheet.worksheet('工作表1')

In [ ]:
def write_to_sheet(date_str, time_str, item, money, category, pay_way):
    """
    接收使用者輸入，並寫入 Google Sheets 的函式。
    """
    try:
        # 驗證日期和時間格式
        datetime.datetime.strptime(date_str, '%Y-%m-%d')
        datetime.datetime.strptime(time_str, '%H:%M')

        # 確保金額是數字
        money = int(money)

        # 建立一個 DataFrame 來儲存單筆資料
        data = pd.DataFrame([{
            '日期': date_str,
            '時間': time_str,
            '品項': item,
            '金額': money,
            '類別': category,
            '支付方式': pay_way
        }])

        # 將 DataFrame 轉為列表以便寫入
        data_to_write = data.values.tolist()

        # 寫入資料到 Google Sheets
        worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')

        return "✅ 資料已成功寫入！"

    except ValueError as ve:
        # 處理格式錯誤
        return f"❌ 格式錯誤：{ve}，請確認日期（YYYY-MM-DD）、時間（HH:MM）或金額的格式是否正確。"
    except Exception as e:
        # 處理其他錯誤
        return f"❌ 寫入失敗：{e}"


In [ ]:
with gr.Blocks(title="支出記錄器") as demo:
    gr.Markdown("## 📋 Google Sheets 支出記錄器")
    gr.Markdown("請在下方輸入您的支出資料，資料將會自動寫入 Google Sheets。")

    with gr.Row():
        date_in = gr.Textbox(label="日期", placeholder="YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
        time_in = gr.Textbox(label="時間", placeholder="HH:MM", value=datetime.datetime.now().strftime("%H:%M"))

    with gr.Row():
        item_in = gr.Textbox(label="品項", placeholder="例如：午餐、車票")
        money_in = gr.Textbox(label="金額", placeholder="數字")

    with gr.Row():
        category_in = gr.Textbox(label="類別", placeholder="例如：外食、交通")
        pay_way_in = gr.Textbox(label="支付方式", placeholder="例如：現金、信用卡")

    submit_btn = gr.Button("寫入資料")
    output_msg = gr.Markdown()

    submit_btn.click(
        fn=write_to_sheet,
        inputs=[date_in, time_in, item_in, money_in, category_in, pay_way_in],
        outputs=output_msg

      )

In [ ]:
demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>